In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json

In [ ]:
# Create the BDS dataset
bds_data = []
analysis_date = datetime(2024, 12, 15)
seq_counter = 1
cluster_sizes = [21, 38, 21, 39, 81]  # Clusters 1-5
total_subjects = sum(cluster_sizes)
# Add summary statistics as separate records
summary_data = []

# Overall R-squared
summary_data.append({
    'USUBJID': 'OVERALL',
    'PARAMCD': 'RSQALL',
    'PARAM': 'Overall R-Squared',
    'AVAL': 0.834832,
    'AVALC': '0.8348',
    'ADT': analysis_date.strftime('%Y-%m-%d'),
    'ASEQ': seq_counter,
    'ANL01FL': 'Y'
})
seq_counter += 1

# Pseudo F Statistic
summary_data.append({
    'USUBJID': 'OVERALL',
    'PARAMCD': 'PSEUDOF',
    'PARAM': 'Pseudo F Statistic',
    'AVAL': 246.40,
    'AVALC': '246.40',
    'ADT': analysis_date.strftime('%Y-%m-%d'),
    'ASEQ': seq_counter,
    'ANL01FL': 'Y'
})
seq_counter += 1

# Cubic Clustering Criterion
summary_data.append({
    'USUBJID': 'OVERALL',
    'PARAMCD': 'CCC',
    'PARAM': 'Cubic Clustering Criterion',
    'AVAL': 2.724,
    'AVALC': '2.724',
    'ADT': analysis_date.strftime('%Y-%m-%d'),
    'ASEQ': seq_counter,
    'ANL01FL': 'Y'
})
seq_counter += 1

# Add cluster-specific statistics
for cluster_num in range(1, 6):
    cluster_freq = cluster_sizes[cluster_num - 1]
    
    # Cluster frequency
    summary_data.append({
        'USUBJID': f'CLUST{cluster_num}',
        'PARAMCD': 'FREQ',
        'PARAM': f'Cluster {cluster_num} Frequency',
        'AVAL': float(cluster_freq),
        'AVALC': str(cluster_freq),
        'ADT': analysis_date.strftime('%Y-%m-%d'),
        'ASEQ': seq_counter,
        'ANL01FL': 'Y'
    })
    seq_counter += 1
    
    # RMS Standard Deviation
    rms_sds = [0.3401, 0.5303, 0.4001, 0.5085, 0.2983]
    summary_data.append({
        'USUBJID': f'CLUST{cluster_num}',
        'PARAMCD': 'RMSSD',
        'PARAM': f'Cluster {cluster_num} RMS Standard Deviation',
        'AVAL': rms_sds[cluster_num - 1],
        'AVALC': f'{rms_sds[cluster_num - 1]:.4f}',
        'ADT': analysis_date.strftime('%Y-%m-%d'),
        'ASEQ': seq_counter,
        'ANL01FL': 'Y'
    })
    seq_counter += 1

In [5]:
# Cluster Means
income_means = [-1.350281302, 1.006673546, -1.348468260, 0.989100984, -0.248824596]
score_means = [1.155830697, -1.222467697, -1.187916616, 1.236400114, -0.013481823]

for cluster_num in range(1, 6):
    summary_data.append({
        'USUBJID': f'CLUST{cluster_num}',
        'PARAMCD': 'MEANINC',
        'PARAM': f'Cluster {cluster_num} Mean Annual Income',
        'AVAL': income_means[cluster_num - 1],
        'AVALC': f'{income_means[cluster_num - 1]:.6f}',
        'ADT': analysis_date.strftime('%Y-%m-%d'),
        'ASEQ': seq_counter,
        'ANL01FL': 'Y'
    })
    seq_counter += 1

    summary_data.append({
        'USUBJID': f'CLUST{cluster_num}',
        'PARAMCD': 'MEANSCOR',
        'PARAM': f'Cluster {cluster_num} Mean Spending Score',
        'AVAL': score_means[cluster_num - 1],
        'AVALC': f'{score_means[cluster_num - 1]:.6f}',
        'ADT': analysis_date.strftime('%Y-%m-%d'),
        'ASEQ': seq_counter,
        'ANL01FL': 'Y'
    })
    seq_counter += 1

# Cluster Standard Deviations
income_sds = [0.2716098169, 0.6195134386, 0.2751574519, 0.6210796936, 0.3353948106]
score_sds = [0.3968968371, 0.4227133289, 0.4944937273, 0.3626340853, 0.2559262296]

for cluster_num in range(1, 6):
    summary_data.append({
        'USUBJID': f'CLUST{cluster_num}',
        'PARAMCD': 'SDINC',
        'PARAM': f'Cluster {cluster_num} SD Annual Income',
        'AVAL': income_sds[cluster_num - 1],
        'AVALC': f'{income_sds[cluster_num - 1]:.10f}',
        'ADT': analysis_date.strftime('%Y-%m-%d'),
        'ASEQ': seq_counter,
        'ANL01FL': 'Y'
    })
    seq_counter += 1

    summary_data.append({
        'USUBJID': f'CLUST{cluster_num}',
        'PARAMCD': 'SDSCOR',
        'PARAM': f'Cluster {cluster_num} SD Spending Score',
        'AVAL': score_sds[cluster_num - 1],
        'AVALC': f'{score_sds[cluster_num - 1]:.10f}',
        'ADT': analysis_date.strftime('%Y-%m-%d'),
        'ASEQ': seq_counter,
        'ANL01FL': 'Y'
    })
    seq_counter += 1

In [ ]:
# Combine all data
df_summary = pd.DataFrame(summary_data)

df_summary

In [ ]:
df_summary = df_summary.sort_values(['USUBJID', 'ASEQ']).reset_index(drop=True)
df_summary

In [9]:
# Save as CSV
csv_filename = "sas_adam_bds_kmeans_clustering.csv"
df_summary.to_csv(csv_filename, index=False)
print(f"\nDataset saved as: {csv_filename}")

# Save as JSON
json_filename = "sas_adam_bds_kmeans_clustering.json"
df_summary.to_json(json_filename, orient='records', indent=2)
print(f"Dataset saved as: {json_filename}")


Dataset saved as: sas_adam_bds_kmeans_clustering.csv
Dataset saved as: sas_adam_bds_kmeans_clustering.json


In [10]:
# Read CSV and convert to list of lists
df = pd.read_csv("./sas_adam_bds_kmeans_clustering.csv")
rows = [
    [
        str(row["USUBJID"]),
        str(row["PARAMCD"]),
        str(row["PARAM"]),
        float(row["AVAL"]) if pd.notna(row["AVAL"]) else None,
        str(row["AVALC"]) if pd.notna(row["AVALC"]) else None,
        str(row["ADT"]),
        int(row["ASEQ"]),
        str(row["ANL01FL"])
    ]
    for _, row in df.iterrows()
]

import json
# Write to JSON file
with open("sas_adam_bds_kmeans_clustering.json", "w") as f:
    json.dump(rows, f, indent=4)